# Sample notebook

軽量なサンプルデータで `src` 配下のコードが動くことを確認します。

In [ ]:
import pandas as pd
import datetime as dt
from src.domain.models import MergedLogDataset
from src.application.use_cases import LoadLogsUseCase
from src.domain.ports import LogRepository

pd.set_option("display.max_columns", 50)

In [ ]:
# サンプルの操作履歴と状態変化を用意
op_df = pd.DataFrame(
    {
        "ContractId": ["A", "A", "B"],
        "OrderReceiptDate": [
            dt.datetime(2023, 1, 1, 10, 0),
            dt.datetime(2023, 1, 1, 11, 0),
            dt.datetime(2023, 1, 1, 9, 30),
        ],
        "FloorCode": ["F1", "F1", "F2"],
        "RoomName": ["101", "101", "202"],
        "EquipmentTypeId": ["AC", "AC", "AC"],
        "EquipmentName": ["エアコン", "エアコン", "エアコン"],
        "PropertyCode": ["mode", "temp", "mode"],
        "PropertyName": ["モード", "温度", "モード"],
        "PropertyValue": ["cool", "25", "heat"],
    }
)

state_df = pd.DataFrame(
    {
        "ContractId": ["A", "A", "B"],
        "ReportedDate": [
            dt.datetime(2023, 1, 1, 10, 5),
            dt.datetime(2023, 1, 1, 11, 2),
            dt.datetime(2023, 1, 1, 9, 40),
        ],
        "FloorCode": ["F1", "F1", "F2"],
        "RoomName": ["101", "101", "202"],
        "EquipmentTypeId": ["AC", "AC", "AC"],
        "EquipmentName": ["エアコン", "エアコン", "エアコン"],
        "PropertyCode1": ["mode", "temp", "mode"],
        "PropertyName1": ["モード", "温度", "モード"],
        "PropertyValue1": ["cool", "25", "heat"],
    }
)


In [ ]:
# ドメインの突合ロジックを直接利用
merged_ds = MergedLogDataset.build_merged_dataset(op_df, state_df, tolerance_minutes=10)
merged_ds.df

In [ ]:
# UseCase 経由での取得を疑似的に試すための簡易リポジトリ
class FakeRepo(LogRepository):
    def __init__(self, op_df: pd.DataFrame, state_df: pd.DataFrame):
        self.op_df = op_df
        self.state_df = state_df

    def fetch_range(self, contract_ids, start_date, end_date=None):
        op = self.op_df.copy()
        st = self.state_df.copy()
        op["OrderReceiptDate"] = pd.to_datetime(op["OrderReceiptDate"], utc=True)
        st["ReportedDate"] = pd.to_datetime(st["ReportedDate"], utc=True)
        op = op[op["ContractId"].isin(contract_ids)]
        st = st[st["ContractId"].isin(contract_ids)]
        start_ts = pd.to_datetime(start_date, utc=True)
        op = op[op["OrderReceiptDate"] >= start_ts]
        st = st[st["ReportedDate"] >= start_ts]
        if end_date:
            end_ts = pd.to_datetime(end_date, utc=True)
            op = op[op["OrderReceiptDate"] <= end_ts]
            st = st[st["ReportedDate"] <= end_ts]
        return op, st


fake_repo = FakeRepo(op_df, state_df)
use_case = LoadLogsUseCase(fake_repo)
merged = use_case.execute(
    contract_ids=["A", "B"], start_date=dt.datetime(2023, 1, 1, 9, 0)
)
merged.df